<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# FAST-OAD Analysis Tutorial
FAST-OAD is a framework for performing rapid Overall Aircraft Design. The computational core of FAST-OAD is based on the  [OpenMDAO framework](https://openmdao.org/).
The GA package stands for General Aviation models.

This notebook will show you the basic step to perform an aircraft candidate analysis based on FAST-OAD features.

To organize our work, we propose to use two user folders `data/` and `workdir/`. In `data/` we store the XML file which describes the aircraft to analyse, here the Beechcraft Duchess. In `workdir/`, we store files generated or modified by FAST-OAD-(GA).

In [1]:
import os.path as pth
import openmdao.api as om
from fastoad import api
from fastga import api as _api
import logging
from fastoad.utils.postprocessing import VariableViewer
import shutil

DATA_FOLDER_PATH = 'data'

WORK_FOLDER_PATH = 'workdir'

# For having log messages on screen
# logging.basicConfig(level=logging.INFO, format='%(levelname)-8s: %(message)s')

# For using all screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

Unable to import mpi4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.


## 1. Read, modify and save different aircraft geometry configurations

In this paragraph we will first run the geometry module on the reference aircraft to derive primary parameters into the detailed geometry set.

**This is done in four steps:**
1. copy the [reference aircraft file](./data/reference_aircraft.xml) to workdir under geometry_inputs.xml name,
2. copy the [geometry.toml file](./data/geometry.toml) to workdir,
3. run process,
4. save output file under the specific name: geometry_reference.xml.

In [2]:
# Copy the reference geometry file (limited input parameters) as input file (name specified in .toml)
shutil.copy(pth.join(DATA_FOLDER_PATH, 'reference_aircraft.xml'), pth.join(WORK_FOLDER_PATH, 'geometry_inputs.xml'))

# Copy the .toml process file to the same location
CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, 'geometry.toml')
shutil.copy(pth.join(DATA_FOLDER_PATH, 'geometry.toml'), CONFIGURATION_FILE)

# Launch an evaluation to obtain the output file (name specified in the .toml)
eval_problem = api.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

# Copy this file to a different name to avoid an overwritte when computing secong geometry
shutil.copy(pth.join(WORK_FOLDER_PATH, 'geometry_outputs.xml'), pth.join(WORK_FOLDER_PATH, 'geometry_reference.xml'))

Error visiting FAST-GA2-MODELS.models.aerodynamics.tests.test_beechcraft_76: Error installing bundle FAST-GA2-MODELS.models.aerodynamics.tests.test_beechcraft_76: No module named 'command'
Error visiting FAST-GA2-MODELS.models.geometry.tests.test_beechcraft_76: Error installing bundle FAST-GA2-MODELS.models.geometry.tests.test_beechcraft_76: No module named 'command'
Error visiting FAST-GA2-MODELS.tests.integration_tests.oad_process.test_oad_process: Error installing bundle FAST-GA2-MODELS.tests.integration_tests.oad_process.test_oad_process: No module named 'command'
Error visiting FAST-GA2-MODELS.utils.physics: Error installing bundle FAST-GA2-MODELS.utils.physics: cannot import name 'Atmosphere2' from 'FAST-GA2-MODELS.utils.physics.atmosphere' (D:\a.reysset\Documents\Github\FAST-GA2-MODELS\utils\physics\atmosphere.py)
Cannot register factory 'test.wrapper.mass_breakdown.beechcraft.dummy_engine' of bundle 203 (FAST-GA2-MODELS.models.weight.mass_breakdown.tests.test_beechcraft_76): 't

'workdir\\geometry_reference.xml'

**This process can be done using the api generate block analysis method to get this working such as a python function:**
1. Same as previous, copy the reference aircraft but already into the aircraft name (same file used all along the process: overwritten)
2. Import the Geometry module
3. Generate a block analysis based on this model
4. Launch a calculation on the wing shape parameters

In [3]:
from fastga.models.geometry import Geometry

# Copy reference aircraft file
AIRCRAFT1_FILE = pth.join(WORK_FOLDER_PATH, 'geometry_long_wing.xml')
shutil.copy(pth.join(DATA_FOLDER_PATH, 'reference_aircraft.xml'), AIRCRAFT1_FILE)

# Define the wing primary geometry parameters name as a list
var_inputs = ["data:geometry:wing:area", "data:geometry:wing:aspect_ratio", "data:geometry:wing:taper_ratio"]

# Declare function
compute_geometry = _api.generate_block_analysis(
        Geometry(propulsion_id="fastga.wrapper.propulsion.basicIC_engine"),
        var_inputs,
        AIRCRAFT1_FILE,
        True,
    )

Cannot register factory 'test.wrapper.handling_qualities.beechcraft.dummy_engine' of bundle 319 (models.handling_qualities.tests.test_beechcraft_76): 'test.wrapper.handling_qualities.beechcraft.dummy_engine' factory already exist
class: <class 'models.handling_qualities.tests.test_beechcraft_76.DummyEngineWrapper'> -- module: models.handling_qualities.tests.test_beechcraft_76
Cannot register factory 'test.wrapper.performances.beechcraft.dummy_engine' of bundle 331 (models.performances.tests.test_beechcraft_76): 'test.wrapper.performances.beechcraft.dummy_engine' factory already exist
class: <class 'models.performances.tests.test_beechcraft_76.DummyEngineWrapper'> -- module: models.performances.tests.test_beechcraft_76
Cannot register factory 'test.wrapper.mass_breakdown.beechcraft.dummy_engine' of bundle 395 (models.weight.mass_breakdown.tests.test_beechcraft_76): 'test.wrapper.mass_breakdown.beechcraft.dummy_engine' factory already exist
class: <class 'models.weight.mass_breakdown.tes

Now, we want to increase the span by 15% without changing the first part of the wing.

We have the following relationships:

$virtualChord=\frac{area}{2*y_{2}+(y_{4}-y_{2})*(1+taperRatio)}$

$aspectRatio=\frac{span^2}{area}$

With $y_{2}$ the y-position of the root chord and $y_{4}$ the y-position of the tip chord.

Meaning:

$\frac{area}{area_{ref}}=\frac{2*y_{2}+(y_{4}-y_{2})*(1+taperRatio)}{2*y_{2,ref}+(y_{4,ref}-y_{2,ref})*(1+taperRatio_{ref})}\implies{}area=\frac{2*y_{2,ref}+(\lambda*y_{4,ref}-y_{2,ref})*(1+taperRatio)}{2*y_{2,ref}+(y_{4,ref}-y_{2,ref})*(1+taperRatio_{ref})}*area_{ref}$

With $\lambda$=1.15.

So, let us read $y_{2}$ and $y_{4}$ in the reference geometry.

In [ ]:
inputs_dict = {"data:geometry:wing:area": (21.66, "m**2"), "data:geometry:wing:aspect_ratio": (9.332, None), "data:geometry:wing:taper_ratio": (0.77, None)}
outputs_dict = compute_geometry(inputs_dict)
print(outputs_dict)

With $y_{2}=0.599$, $y_{4}=6.182$ we find:

In [ ]:
taper_func = lambda taper_old, lamb: lamb * taper_old + (1 - lamb)
area_func = lambda area_ref, taper_ratio_old, taper_ratio_new, lamb, y2_ref, y4_ref : (2*y2_ref + (lamb*y4_ref-y2_ref)*(1+taper_ratio_new))/(2*y2_ref + (y4_ref-y2_ref)*(1+taper_ratio_old)) * area_ref
taper_ratio_new = taper_func(0.8, 1.15)
area = area_func(19.151, 0.8, taper_ratio_new, 1.15, 0.599, 6.182)
span = 12.363014640450768
print("area=" + str(area))
print("aspect_ratio=" + str((span*1.15)**2/area))
print("taper_ratio=" + str(taper_ratio_new))

So we will apply the modifications of aspect ratio and area in accordance with previous results:

In [ ]:
api.variable_viewer(pth.join(WORK_FOLDER_PATH, 'geometry_inputs.xml'))

We will finish this step by saving the output generated geometry under [geometry_long_wing.xml](./workdir/geometry_long_wing.xml) file name.

In [ ]:
eval_problem = api.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

shutil.copy(pth.join(WORK_FOLDER_PATH, 'geometry_outputs.xml'), pth.join(WORK_FOLDER_PATH, 'geometry_long_wing.xml'))

We will finish by visualizing the two generated geometries before performing other calculation/analysis steps.

In [ ]:
from fastoad.utils.postprocessing.analysis_and_plots import aircraft_geometry_plot

fig = aircraft_geometry_plot(pth.join(WORK_FOLDER_PATH, 'geometry_reference.xml'), name='reference aircraft')
fig = aircraft_geometry_plot(pth.join(WORK_FOLDER_PATH, 'geometry_long_wing.xml'), name='long wing aircraft', fig=fig)
fig.show()

## 2. Plotting evolution diagram of the aircrafts
In this chapter we will see how to plot evolution diagram for the two aircraft geometries presented above.

## 3. Mass breakdown plots

This first mass breakdown plot enables to visualize the Maximum TakeOff Weight (MTOW) and Overall Weight Empty (OWE) for a single design.

In [ ]:
fig = mass_breakdown_sun_plot(Beechcraft_800nm_MDA_OUTPUT_FILE)
fig.show()

This second mass breakdown plot provides less detail but enables to compare designs.

In [ ]:
fig = mass_breakdown_bar_plot(Beechcraft_800nm_MDA_OUTPUT_FILE, name='Beechcraft 800 nm MDA')
fig = mass_breakdown_bar_plot(Beechcraft_1000nm_MDA_OUTPUT_FILE, name='Beechcraft 1000 nm MDA', fig=fig)
#fig = mass_breakdown_bar_plot(Beechcraft_800nm_MDO_OUTPUT_FILE, name='Beechcraft 800 nm MDO', fig=fig)
fig.show()